In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE94524"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE94524"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE94524.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE94524.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE94524.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tamoxifen-associated endometrial tumors expose differential enhancer activity for Estrogen Receptor alpha"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: endometrioid adenocarcinoma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analysis and decisions:

# 1. Gene Expression Data Availability
# Based on the SuperSeries description, it's likely to contain gene expression data as part of its SubSeries
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Reviewing the sample characteristics dictionary
# For trait: We can see 'tissue: endometrioid adenocarcinoma' in row 0
# This indicates the tissue type, which can be used as the trait information
trait_row = 0

# For age: There is no information about age in the sample characteristics
age_row = None

# For gender: There is no information about gender in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# For trait:
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if value indicates endometrioid cancer
    if "endometrioid" in value.lower() and "adenocarcinoma" in value.lower():
        return 1  # Indicates endometrioid cancer
    else:
        return 0  # Not endometrioid cancer

# Since age and gender are not available, we'll define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait availability based on trait_row
is_trait_available = trait_row is not None

# Save initial cohort info
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Check if trait data is available
if trait_row is not None:
    # Define the path to the expected clinical data file
    clinical_file_path = f"{in_cohort_dir}/clinical.csv"
    
    # Check if the file exists before trying to load it
    if os.path.exists(clinical_file_path):
        try:
            clinical_data = pd.read_csv(clinical_file_path)
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the resulting dataframe
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Save the clinical data to the specified path
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        except Exception as e:
            print(f"Error processing clinical data: {e}")
    else:
        print(f"Clinical data file not found at {clinical_file_path}. Skipping clinical feature extraction.")
else:
    print("No trait data available, skipping clinical feature extraction.")


Clinical data file not found at ../../input/GEO/Endometrioid_Cancer/GSE94524/clinical.csv. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 73
Header line: "ID_REF"	"GSM2477471"	"GSM2477472"	"GSM2477473"	"GSM2477474"	"GSM2477475"	"GSM2477476"	"GSM2477477"	"GSM2477478"	"GSM2477479"	"GSM2477480"	"GSM2477481"	"GSM2477482"	"GSM2477483"	"GSM2477484"	"GSM2477485"	"GSM2477486"	"GSM2477487"	"GSM2477488"	"GSM2477489"	"GSM2477490"	"GSM2477491"	"GSM2477492"	"GSM2477493"	"GSM2477494"	"GSM2477495"	"GSM2477496"	"GSM2477497"	"GSM2477498"	"GSM2477499"	"GSM2477500"	"GSM2477501"	"GSM2477502"	"GSM2477503"	"GSM2477504"	"GSM2477505"	"GSM2477506"	"GSM2477507"	"GSM2477508"	"GSM2477509"	"GSM2477510"	"GSM2477511"	"GSM2477512"	"GSM2477513"	"GSM2477514"	"GSM2477515"	"GSM2477516"	"GSM2477517"	"GSM2477518"	"GSM2477519"	"GSM2477520"	"GSM2477521"	"GSM2477522"	"GSM2477523"	"GSM2477524"	"GSM2477525"	"GSM2477526"	"GSM2477527"	"GSM2477528"	"GSM2477529"	"GSM2477530"	"GSM2477531"	"GSM2477532"	"GSM2477533"	"GSM2477534"	"GSM2477535"	"GSM2477536"	"GSM2477537"	"GSM2477538"	"GSM2477539"	"GSM2477540"	"GSM2477541"	"GSM2477542"	"GSM2477543"	

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers in the gene expression data
# Looking at the first line of data, we see numeric identifiers (1, 2, 3, etc.)
# These appear to be numeric probe IDs and not human gene symbols
# Typically human gene symbols would be alphanumeric identifiers like "BRCA1", "TP53", etc.
# Therefore, these identifiers need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'MetaRow': ['12', '12', '12', '12', '12'], 'MetaCol': ['4', '4', '4', '4', '4'], 'SubRow': ['28', '27', '26', '25', '24'], 'SubCol': [28.0, 28.0, 28.0, 28.0, 28.0], 'Reporter ID': [334575.0, 333055.0, 331915.0, 330395.0, 328875.0], 'oligo_id': ['H300009761', 'H300009722', 'H300000470', 'H300000646', 'H300004276'], 'oligo_type': ['I', 'I', 'I', 'I', 'I'], 'gene_id': ['ENSG00000182037', 'ENSG00000180563', 'ENSG00000179449', 'ENSG00000177996', 'ENSG00000176539'], 'transcript_count': [1.0, 1.0, 1.0, 1.0, 1.0], 'representative_transcript_id': ['ENST00000315389', 'ENST00000316343', 'ENST00000314233', 'ENST00000325950', 'ENST00000326170'], 'HUGO': [nan, nan, 'MAGEL2', nan, nan], 'GB_LIST': [nan, nan, 'NM_019066, AF200625', nan, nan], 'GI-Bacillus': [nan, nan, nan, nan, nan], 'SPOT_ID': ['ENSG00000182037', 'ENSG00000180563', nan, 'ENSG00000177996', 'ENSG00000176539'], 'SEQUENCE': ['TTAATCTGACCTGTGAAAAACACTGTCCAGAGGCTAGGTGCGGTGGCTAACGCT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation dataframe to identify appropriate columns for mapping
# The 'ID' column in gene_annotation contains numeric IDs matching the gene expression data indices
# The 'HUGO' column appears to contain gene symbols, which is what we need for mapping

# 2. Generate the gene mapping dataframe using the library function
# This is more efficient than processing row by row
print("Creating gene mapping from available annotation data")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='HUGO')

# Check the mapping size
print(f"Created mapping with {len(mapping_df)} entries")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("Applying gene mapping to convert probe data to gene expression data...")
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to ensure consistency
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)

# Preview the resulting gene expression data
print("\nPreview of gene expression data after mapping:")
print(gene_data.head(5))
print(f"Shape of gene expression data: {gene_data.shape}")

# Save the processed gene expression data
if not os.path.exists(os.path.dirname(out_gene_data_file)):
    os.makedirs(os.path.dirname(out_gene_data_file))
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping from available annotation data
Created mapping with 13569 entries
Applying gene mapping to convert probe data to gene expression data...


Normalizing gene symbols...

Preview of gene expression data after mapping:
        GSM2477471  GSM2477472  GSM2477473  GSM2477474  GSM2477475  \
Gene                                                                 
A1BG      0.152441   -0.328190   -0.402093   -0.459311   -0.044677   
A2M       0.194438   -1.377660   -0.561084    1.047960    0.589405   
A4GALT    1.089600    0.343792    0.053120    0.364721    0.088068   
AAAS     -0.532202    0.709348   -0.419568    0.101501   -0.501564   
AADAC     0.282887   -1.911730    0.000000    3.241060    0.711206   

        GSM2477476  GSM2477477  GSM2477478  GSM2477479  GSM2477480  ...  \
Gene                                                                ...   
A1BG      0.990185    0.305397    0.038350   -0.176839   -0.323575  ...   
A2M      -1.436880   -0.413543   -0.884551    0.254121    0.984051  ...   
A4GALT   -0.122575    0.008176    0.927995   -0.029103    0.440727  ...   
AAAS     -0.374575   -0.298828   -0.494604    0.080157    

Gene expression data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE94524.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (9628, 111)
First few genes with their expression values after normalization:
        GSM2477471  GSM2477472  GSM2477473  GSM2477474  GSM2477475  \
Gene                                                                 
A1BG      0.152441   -0.328190   -0.402093   -0.459311   -0.044677   
A2M       0.194438   -1.377660   -0.561084    1.047960    0.589405   
A4GALT    1.089600    0.343792    0.053120    0.364721    0.088068   
AAAS     -0.532202    0.709348   -0.419568    0.101501   -0.501564   
AADAC     0.282887   -1.911730    0.000000    3.241060    0.711206   

        GSM2477476  GSM2477477  GSM2477478  GSM2477479  GSM2477480  ...  \
Gene                                                                ...   
A1BG      0.990185    0.305397    0.038350   -0.176839   -0.323575  ...   
A2M      -1.436880   -0.413543   -0.884551    0.254121    0.984051  ...   
A4GALT   -0.122575    0.008176    0.927995   -0.029103    0.440727  ...   
AAAS     -0.374575   -0.2988

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE94524.csv
Raw clinical data shape: (1, 112)
Clinical features:
                     GSM2477471  GSM2477472  GSM2477473  GSM2477474  \
Endometrioid_Cancer         1.0         1.0         1.0         1.0   

                     GSM2477475  GSM2477476  GSM2477477  GSM2477478  \
Endometrioid_Cancer         1.0         1.0         1.0         1.0   

                     GSM2477479  GSM2477480  ...  GSM2477572  GSM2477573  \
Endometrioid_Cancer         1.0         1.0  ...         1.0         1.0   

                     GSM2477574  GSM2477575  GSM2477576  GSM2477577  \
Endometrioid_Cancer         1.0         1.0         1.0         1.0   

                     GSM2477578  GSM2477579  GSM2477580  GSM2477581  
Endometrioid_Cancer         1.0         1.0         1.0         1.0  

[1 rows x 111 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE94524.csv
Linked

Data shape after handling missing values: (111, 9629)
Quartiles for 'Endometrioid_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Endometrioid_Cancer' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
